<a href="https://colab.research.google.com/github/jashwanth1818/Fmml-labs/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

The size of the validation set in machine learning can have a significant impact on model training and evaluation. Here's how the accuracy of the validation set is affected when you increase or reduce its percentage:

   1. Positive Impact on Validation Accuracy: When you increase the percentage of the validation set, you typically allocate more data for evaluating the model's performance. This can lead to a more accurate assessment of the model's generalization capabilities.

   2. Reduced Training Data: However, as you allocate more data to the validation set, you have less data available for training. This can be problematic if your training dataset is already small. Smaller training datasets may result in models that underfit because they lack sufficient examples to learn from.

   3. Risk of Overfitting: With a larger validation set, models may generalize better to the validation data, but they might not perform as well on unseen, real-world data if the training dataset is too small. There's also a risk that you'll overfit to the validation set itself.

   4. More Data for Training: Reducing the percentage of the validation set means more data is available for training. This can be beneficial, especially when you have a limited dataset, as models tend to perform better with more training examples.

   5. Risk of Overfitting the Validation Set: A smaller validation set may not effectively capture the model's true generalization performance. It's easier for the model to perform well on a small validation set by chance, which can lead to overly optimistic estimates of performance.

   6. Uncertainty in Evaluation: With a smaller validation set, the accuracy scores may have higher variance, making it harder to confidently assess the model's performance.
The size of the train and validation sets can affect the predictiveness of accuracy on the test set using the validation set as follows:

Large Train Set, Small Validation Set: If the training set is much larger than the validation set, the model may have ample data to learn from but not enough to effectively validate its performance. This situation could lead to overfitting on the training data, where the model memorizes the training examples but fails to generalize well to unseen data. The validation set's accuracy may not be a reliable predictor of the test set's accuracy because the model might perform well on the small validation set by chance.

Balanced Train and Validation Set Sizes: Having a roughly balanced size between the training and validation sets allows for better model evaluation. The validation set can provide a reasonably accurate estimate of the model's generalization performance on unseen data, which is a good predictor of how it will perform on the test set.

Large Validation Set, Small Train Set: If the validation set is significantly larger than the training set, the model may perform well on the validation set, but this performance may not generalize to the test set. This imbalance can lead to underfitting because the model does not have sufficient training data to learn complex patterns in the data.

Appropriate Cross-Validation: Techniques like cross-validation can help mitigate the impact of varying train and validation set sizes.

A common practice in machine learning is to reserve around 20-30% of your dataset for the validation set to strike a reasonable balance between having enough data for training and reliable model evaluation. However, the ideal percentage can vary depending on factors like the size of your dataset and the nature of your problem. Here are some rough guidelines:

Small Datasets: If you have a small dataset (e.g., less than a few thousand examples), you may want to allocate a larger portion, such as 30%, to the validation set to ensure a reasonably sized training set.

Large Datasets: For larger datasets, you can often get away with a smaller validation set, around 20%, because you still have a substantial amount of data for training, and the validation set can provide a reliable estimate of model performance.

Cross-Validation: In situations where you have limited data or need a more robust evaluation, techniques like k-fold cross-validation can be employed. In this case, the validation set size is less critical because you're repeatedly splitting the data into multiple validation sets.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits can give more consistent results. By performing multiple splits and averaging the validation accuracy, we can reduce the impact of any specific split's randomness or variability. This helps to provide a more robust and reliable estimate of the model's performance.

Yes, averaging the validation accuracy across multiple splits can provide a more accurate estimate of the test accuracy. By considering multiple splits, we can account for variations in the data and get a better understanding of how the model performs on unseen data. It helps to reduce the bias and provides a more reliable estimate of the model's generalization ability.

The number of iterations can have an impact on the estimate of the model's performance. Generally, with higher iterations, we might get a better estimate of the model's accuracy. More iterations allow the model to learn and adjust its parameters, potentially improving its performance. However, it's important to find the right balance, as too many iterations can lead to overfitting and a decrease in generalization ability.

Yes, increasing the iterations can help mitigate the impact of a small train or validation dataset. By running more iterations, the model has more opportunities to learn from the available data and make better predictions. It can help compensate for the limited amount of data and potentially improve the model's performance. However, it's still important to have a representative dataset and avoid overfitting.